In [30]:
from torchvision import transforms
import numpy as np
import torch
import torchvision
import torch.nn.functional as F
from torch import nn
from torch import optim
from PIL import Image

In [ ]:
torch.__version__

### 分析数据


In [1]:
#准备数据集，其中0.1307，0.3081为MNIST数据的均值和标准差，这样操作能够对其进行标准化
#因为MNIST只有一个通道（黑白图片）,所以元组中只有一个值
dataset = torchvision.datasets.MNIST('/data', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

dataset.data.size(), dataset.data[0], 

NameError: name 'torchvision' is not defined

In [34]:
Image.fromarray(dataset.data[1].numpy())

### 构建模型

In [37]:
x = dataset.data
print(x.size())
x = x.view(-1, 28*28*1)
x.size()

torch.Size([60000, 28, 28])


torch.Size([60000, 784])

In [23]:
class MnistNet(nn.Module):
    def __init__(self):
        super(MnistNet,self).__init__()
        self.fc1 = nn.Linear(28*28*1,28)  #定义Linear的输入和输出的形状
        self.fc2 = nn.Linear(28,10)  #定义Linear的输入和输出的形状

    def forward(self,x):
        x = x.view(-1,28*28*1)  #对数据形状变形，-1表示该位置根据后面的形状自动调整
        x = self.fc1(x) #[batch_size,28]
        x = F.relu(x)  #[batch_size,28]
        x = self.fc2(x) #[batch_size,10]
        return F.log_softmax(x, dim=-1)

### 准备数据迭代器

In [ ]:
train_dataloader = torch.utils.data.DataLoader(dataset,batch_size=64,shuffle=True)

In [ ]:
train_batch_size = 200
test_batch_size = 1000
img_size = 28

def get_dataloader(train=True):

    #准备数据集，其中0.1307，0.3081为MNIST数据的均值和标准差，这样操作能够对其进行标准化
    #因为MNIST只有一个通道（黑白图片）,所以元组中只有一个值
    dataset = torchvision.datasets.MNIST('/data', train=train, download=True,
                                         transform=torchvision.transforms.Compose([
                                         torchvision.transforms.ToTensor(),
                                         torchvision.transforms.Normalize((0.1307,), (0.3081,)),]))
    #准备数据迭代器
    batch_size = train_batch_size if train else test_batch_size
    dataloader = torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=True)
    return dataloader

### 训练

In [24]:
mnist_net = MnistNet()
mnist_net = mnist_net.cuda()
optimizer = optim.Adam(mnist_net.parameters(),lr= 0.001)

def train(epoch):
    mnist_net.train(mode=True) #模型设置为训练模型
    
    train_dataloader = get_dataloader(train=True) #获取训练数据集
    for idx,(data,target) in enumerate(train_dataloader):
        optimizer.zero_grad() #梯度置为0
        data = data.cuda()
        target = target.cuda()
        output = mnist_net(data) #进行向前计算
      
        loss = F.nll_loss(output,target) #带权损失
        loss.backward()  #进行反向传播，计算梯度
        optimizer.step() #参数更新
        if idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, idx * len(data), len(train_dataloader.dataset),
                       100. * idx / len(train_dataloader), loss.item()))

In [25]:
for i in range(5):
    train(i)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.354496
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.351051
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.242459
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.180571
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.226569
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.185199
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.214285
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.221121
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.193865
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.226167
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.199483
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.129448
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.144224
Train Epoch: 4 [20000/60000 (33%)]	Loss: 0.222717
Train Epoch: 4 [40000/60000 (67%)]	Loss: 0.131298


### 测试

In [9]:
def test():
    test_loss = 0
    correct = 0
    mnist_net.eval()
    test_dataloader = get_dataloader(train=False)
    with torch.no_grad():
        for data, target in test_dataloader:
            data = data.cuda()
            target = target.cuda()
            output = mnist_net(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1] #获取最大值的位置,[batch_size,1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_dataloader.dataset)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_dataloader.dataset),
        100. * correct / len(test_dataloader.dataset)))

In [10]:
test()


Test set: Avg. loss: 0.1328, Accuracy: 9616/10000 (96.16%)

